# NVIDIA Power Draw

**Note:** this notebook is meant to be executed on the supplied Docker image.

In [ ]:
import time
import subprocess

In order to read current power draw of an NVIDIA graphics card, `nvidia-smi` is used. The command `nvidia-smi --query-gpu=power.draw --format=csv` returns the current power draw in CSV format. The following function (`power_draw_w`) calls `nvidia-smi` and reads the numerical value of current power draw.

In [ ]:
def power_draw_w():
    proc = subprocess.run(
        ["nvidia-smi", "--query-gpu=power.draw", "--format=csv"],
        capture_output=True)
    stdout = proc.stdout.decode('utf-8')
    return float(stdout.split('\n')[1].split(' ')[0])

print(f'Current power draw: {power_draw_w()} W')

Measuring energy consumption on NVIDIA graphics cards involves integrating the power draw over time. The following function (`measure_power`) runs a program in background and calculates energy consumption during its execution. For each time interval (the length of which is set using the parameter `resolution`), consumed energy is calculated as `power_draw * time`.

In [ ]:
def measure_power(args, resolution=0.5, log=True):
    NS_IN_S = 1_000_000_000

    energy_j = 0
    proc = subprocess.Popen(args)
    start_time_ns = time.time_ns()
    time_ns = start_time_ns
    
    while True:
        timeout = False
        try:
            proc.wait(timeout=resolution)
        except subprocess.TimeoutExpired:
            timeout = True
        new_time_ns = time.time_ns()
        draw_w = power_draw_w()
        if log:
            print(f'Draw at {(new_time_ns - start_time_ns) / NS_IN_S} s: {draw_w} W')
        
        delay_ns = new_time_ns - time_ns
        energy_j += delay_ns * draw_w / NS_IN_S
        time_ns = new_time_ns
        
        if not timeout:
            break
    
    total_time_s = (time_ns - start_time_ns) / NS_IN_S
    if log:
        print(f'Total energy consumed: {energy_j} J')
        print(f'Average draw: {energy_j / total_time_s} W')

By executing this function with command `sleep 10` we can measure idle energy consumption over the interval of 10 seconds.

In [ ]:
measure_power(['sleep', '10'], resolution=0.5)

You can change the command to run any other application and measure its energy consumption of the GPU.

The following code will measure energy consumption of a script named `resnet152predict`, which uses the ResNet-152 network to predict the contents of images. A sample set of images is provided inside the directory `/images/`.

In [ ]:
# first, make sure that the model is downloaded before measuring energy
subprocess.run(['resnet152predict', '/images/'],
    stdout = subprocess.DEVNULL,
    stderr = subprocess.DEVNULL)

measure_power(['resnet152predict', '/images/'], resolution=0.5)